# HealthOmics ECR Setup
The intent of this guide is to help individuals who want to setup a brand new AWS account for AWS HealthOmics.

#### Prerequisites 
+ AWS Account 
+ manifest.json file to build ECR repositories

#### Requirements: 
+ AWS CDK 
+ AWS CLI 
Due to security policy, the easiest way to perform the following steps will be through the AWS Console's Cloud Shell. If you do have both CDK and AWS cli tool setup in your local environment, you can run it against the desire AWS profile as well. 

#### Open Terminal and run following command to install CDK
<p style="background:black">
<code style="background:black;color:white">npm install -g aws-cdk
</code>
</p>

#### Import relevant libraries

In [ ]:
import json
from datetime import datetime
import os
from time import sleep
from urllib.parse import urlparse
from zipfile import ZipFile, ZIP_DEFLATED

import boto3
import botocore.exceptions

## Step 1: Setup S3 bucket 
HealthOmics will requires an S3 bucket to store the run logs and output files. <br>
You can create an S3 bucket through AWS console **or** run the following terminal command:

In [ ]:
!aws s3 mb s3://[bucket-name]

In [ ]:
bucket_name = "nigms-scrnaseq-bucket-demo-2"

In [ ]:
account_id = boto3.client('sts').get_caller_identity().get('Account')

In [ ]:
region = boto3.session.Session().region_name

In [ ]:
account_id

In [ ]:
!aws s3 mb s3://$bucket_name

<div class="alert alert-block alert-info">
<b>NOTE:</b> Replace [bucket-name] with the name of your S3 bucket.</div>

## Step 2: Create HealthOmics Policy and attach to default SageMaker Role
When creating your notebook instance make sure to attach IAM policies that enable the creation of new IAM roles. Failing to attach the correct IAM-based policies may result in error when creating and attaching new roles or policies.

In [ ]:
# Define demo policies
omics_demo_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject"
            ],
            "Resource": [
                "arn:aws:s3:::"+bucket_name+"/*",
                "arn:aws:s3:::aws-genomics-static-us-east-1/workflow_migration_workshop/nfcore-scrnaseq-v2.3.0/*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3::::"+bucket_name,
                "arn:aws:s3:::aws-genomics-static-us-east-1/workflow_migration_workshop/nfcore-scrnaseq-v2.3.0"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:PutObject"
            ],
            "Resource": [
                "arn:aws:s3:::"+bucket_name+"/*",
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:DescribeLogStreams",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": [
                "arn:aws:logs:"+region+":"+account_id+":log-group:/aws/omics/WorkflowLog:log-stream:*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup"
            ],
            "Resource": [
                "arn:aws:logs:"+region+":"+account_id+":log-group:/aws/omics/WorkflowLog:*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "ecr:BatchGetImage",
                "ecr:GetDownloadUrlForLayer",
                "ecr:BatchCheckLayerAvailability"
            ],
            "Resource": [
                "arn:aws:ecr:"+region+":"+account_id+":repository/*"
            ]
        }
    ]
}

omics_demo_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "omics.amazonaws.com"
            },
            "Action": "sts:AssumeRole",
            "Condition": {
                "StringEquals": {
                    "aws:SourceAccount": account_id
                },
                "ArnLike": {
                    "aws:SourceArn": "arn:aws:omics:"+region+":"+account_id+":run/*"
                }
            }
        }
    ]
}

### Attach new policy to default role that was established when instance was created

In [ ]:
# We will use this as the base name for our role and policy
omics_iam_name = 'SageMaker_HealthOmics_test_john'

# Create the iam client
iam = boto3.resource('iam')

# Check if the role already exists; if not, create it

role = iam.Role(omics_iam_name)
try:
    role.load()
except iam.meta.client.exceptions.NoSuchEntityException:
    create_role_response = iam.create_role(
        RoleName = omics_iam_name,
        AssumeRolePolicyDocument = json.dumps(omics_demo_trust_policy)
    )
    role.load()
policy = iam.Policy('arn:aws:iam::'+account_id+':policy/{}-policy'.format(omics_iam_name))

try:
    policy.load()
except iam.meta.client.exceptions.NoSuchEntityException:
    #Create policy
    policy = iam.create_policy(
        PolicyName='{}-policy'.format(omics_iam_name),
        Description="Policy for AWS HealthOmics demo",
        PolicyDocument=json.dumps(omics_demo_policy)
    )
    policy.load()

#Attach the policy to the role
policy.attach_role(RoleName=omics_iam_name)
print(f'New policy for {omics_iam_name} successfully attached')

<div class="alert alert-block alert-info">
<b>NOTE:</b> Replace [Role Name] name of attached role when you created the notebook instance.</div>

## Step 3: Setup ECR for HealthOmics
In this section we will go through the everything required to convert pre-existing nextflow script into HealthOmics workflow. Containers used by workflow will have to be fetched from ECR. If the AWS account already has ECR repository with necessary container image(s) then can skip this part.

#### Clone omx-ecr-helper Github repository

In [ ]:
#Make sure you're within the Sagemaker directory and list 
!pwd
!ls

In [ ]:
#clone the omx-ecr-helper github repository into current directory
!git clone https://github.com/CBIIT/omx-ecr-helper

In [ ]:
#You should now see the 'omx-ecr-helper' folder within the directory when running ls command
!ls

#### Now open terminal and run following commands to install CDK
##### Bootstrap CDK
<p style="background:black">
<code style="background:black;color:white">cdk bootstrap aws://[AWS-ACCOUNT-NUMBER]/[AWS-REGION]
</code>
</p>

<div class="alert alert-block alert-info">
<b>NOTE:</b> Replace [AWS-ACCOUNT-NUMBER] and [AWS-REGION] with appropriate values.</div>

##### Change your working dir to the repo: 
<p style="background:black">
<code style="background:black;color:white">cd omx-ecr-helper
</code>
</p>

##### Install all the dependencies: 
<p style="background:black">
<code style="background:black;color:white">npm install
</code>
</p> 

##### Deploy the CF stacks for ECR in your account: 
<p style="background:black">
<code style="background:black;color:white">cdk deploy --all --require-approval never 
</code>
</p>

### Clone and setup nf-core scrnaseq Github repository

In [ ]:
#clone nf-core repository
!git clone https://github.com/nf-core/scrnaseq.git --branch 2.3.0 --single-branch

In [ ]:
#copy latest version of docker manifest file from helper repo
!cp ./omx-ecr-helper/lib/lambda/parse-image-uri/public_registry_properties.json namespace.config

In [ ]:
#clone amazon-omics-tutorials repo to get inspect_nf.py script
!git clone https://github.com/aws-samples/amazon-omics-tutorials.git

### Generate manifest.json and omics.config files

In [ ]:
#run inspect_nf.py to create manifest.json and omics.config files
!python3 amazon-omics-tutorials/utils/scripts/inspect_nf.py \
--output-manifest-file scrnaseq/scrnaseq_230_docker_image_manifest.json \
-n scrnaseq/namespace.config \
--output-config-file scrnaseq/conf/omics.config \
--region $region \
scrnaseq/

##### Execute the StepFunctions
After running the code below, the container images listed in your 'manifest.json' file should now be present within your account's ECR.

In [ ]:
#us-east-1:664418964547
!aws stepfunctions start-execution\
    --state-machine-arn arn:aws:states:[AWS REGION]:[AWS-ACCOUNT-NUMBER]:stateMachine:omx-container-puller\
    --input file://container_pull_manifest.json

In [ ]:
#us-east-1:664418964547
!aws stepfunctions start-execution\
    --state-machine-arn arn:aws:states:$region:$account_id:stateMachine:omx-container-puller\
    --input file://scrnaseq/scrnaseq_230_docker_image_manifest.json

<div class="alert alert-block alert-info">
<b>NOTE:</b> Replace [AWS-ACCOUNT-NUMBER] and [AWS-REGION] with appropriate values.</div>

In [ ]:
#print new container repos, may print a lot of text
!aws ecr describe-repositories

In [ ]:
#move omcis.config file to project conf directory
! mv omics.config scrnaseq/conf

In [ ]:
#update nextflow.config file to include newly created omics.config
!echo "includeConfig 'conf/omics.config'" >> scrnaseq/nextflow.config

### Now you should be ready to setup your data stores, create workflows, connect the workflows to your account's ECR, and start running your workflows!